In [1]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install meteostat

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import optuna
import gc
import requests
import json
from sklearn.preprocessing import SplineTransformer
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error,r2_score,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,KFold
from datetime import date
from datetime import datetime
from sklearn.cluster import KMeans
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import os
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 50)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [ ]:
!pip install importnb


In [ ]:
# with the new api
from importnb import imports
with imports("ipynb"):
    import modules



In [ ]:
class CFG:
    target = 'tüketim'
    train_path = 'data/train.csv'
    test_path = 'data/sample_submission.csv'
    med_path = 'data/med.csv'
    submission_path='data/sample_submission.csv'
    generation_path='data/TR_enerji_uretimi.csv'
    weatherfrom_wwo='data/izmir_weather_data.csv'
    meteostat_feature_columns = ['temp','dwpt','rhum','prcp','wdir','wspd','pres','coco']
    production_features = True
    consumption_features = False
    yuk_tahmin_plani_features = False

### Read Data

In [ ]:
train=pd.read_csv(CFG.train_path).rename(columns={'Dağıtılan Enerji (MWh)':'tüketim'})
test=pd.read_csv(CFG.test_path).rename(columns={'Dağıtılan Enerji (MWh)':'tüketim'})
med=pd.read_csv(CFG.med_path)
submission=pd.read_csv(CFG.submission_path)
train = modules.timeseries_features(train.set_index('Tarih'))
test= modules.timeseries_features(test.set_index('Tarih'))
test['tüketim']=test['tüketim'].replace(0,np.nan)
train.index=pd.to_datetime(train.index)

### Train and Test data merged

In [ ]:
all_data=pd.concat([train,test],axis=0)
all_data.index=pd.to_datetime(all_data.index)
med.Tarih=pd.to_datetime(med.Tarih)
# Gün içindeki elektrik kesinti süresinin kabul edilebilir limiti aştığı gün.
med['Tarih']=med['Tarih'].astype(str)

## External Data

In [ ]:
weather=modules.get_meteostat_data(datetime(2018, 1, 1), datetime(2022, 9, 1),features=CFG.meteostat_feature_columns)[:-1]
generation=pd.read_csv(CFG.generation_path).iloc[:,1:]
generation.Tarih=pd.to_datetime(generation.Tarih)
generation=generation[['total','Tarih']]
# generation=real_time_generation()
consumption=modules.real_time_consumption()
con_pro=consumption.merge(generation,on='Tarih',how='left')
con_pro.set_index('Tarih',drop=True,inplace=True)
all_data=all_data.merge(weather,left_index=True,right_index=True)
all_data=all_data.merge(con_pro,left_index=True,right_index=True)

# ------------------------------------------------------------------------------------------------------------------------------------
yuk=pd.read_csv('data/YukTahmin.csv',encoding='unicode_escape').iloc[:40896,2]
all_data['yuk']=yuk.values
all_data.yuk=all_data.yuk.str.replace(',','').astype(float)
# ------------------------------------------------------------------------------------------------------------------------------------
ep=pd.read_csv('data/DENGESIZLIK.csv',encoding='unicode_escape').iloc[:40896,2:]
all_data['pozitif']=ep['Pozitif Dengesizlik Miktarý (MWh)'].values
all_data['pozitif']=all_data['pozitif'].str.replace(',','').astype(float)
# ------------------------------------------------------------------------------------------------------------------------------------
ue=pd.read_csv('data/UECM.csv',encoding='unicode_escape').iloc[:40896,2:]
ue['UEÇM (MWh)']=ue['UEÇM (MWh)'].str.replace(',','').astype(float)
all_data['ue']=ue['UEÇM (MWh)'].values

# ------------------------------------------------------------------------------------------------------------------------------------
weatherfrom_wwo=pd.read_csv(CFG.weatherfrom_wwo,index_col='date_time')[:'2022-08-31 23:00:00']
weatherfrom_wwo.index=pd.to_datetime(weatherfrom_wwo.index)
all_data=all_data.merge(weatherfrom_wwo[['pressure','humidity','uvIndex','WindGustKmph','windspeedKmph']],left_index=True,right_index=True)
all_index=all_data.index

### Lag Features

In [ ]:
all_data.columns

In [ ]:
all_data=modules.lag_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],48,24) 
# all_data=diff_pct_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue'],1) 
all_data=modules.rolling_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],[6,12,18,24,30,36,42,48],['mean',''])
all_data=modules.rolling_shift_features(all_data,['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],[6,12,18,24,30,36,42,48],['mean','std'],shift=24)

# hour_df=seasonality_spline_features()
# all_data=all_data.merge(hour_df, on='hour').set_index(all_data.index)

###  Changed some columns of dtypes to category

In [ ]:
all_data['med_label']=np.where(all_data.date.isin(med.Tarih),1,0)
all_data=modules.is_categorical(all_data,all_data.columns)

In [ ]:
df_train=all_data[all_data.tüketim.notnull()]
df_test=all_data[all_data.tüketim.isnull()]
print(df_train.shape)
print(df_test.shape)

### Target Encoding by some of seasonal columns

In [ ]:
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim','hour','mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekend','hour'],'mean')
df_train,df_test=modules.cat_target_encode(df_train,df_test,'tüketim',['season','weekday','prime_time','hour'],'mean')
all_data=pd.concat([df_train,df_test])
all_data.drop(['pres','rhum','humidity','uvIndex','WindGustKmph','temp','total','yuk','ue','pozitif'],axis=1,inplace=True)


In [ ]:
# all_data=all_data.reset_index(drop=True)
all_data.drop([
'date',
'dayofyear',
'dayofweek',
 # 'weekofyear'
# 'year'
],axis=1,inplace=True)

### Categorical features are labeled for xgboost

In [ ]:
cat_features=all_data.select_dtypes('category').columns
le_fit={}
all_data[cat_features],le_fit=modules.label_data(feature=cat_features,data=all_data,le_fit=le_fit)
data_train=all_data[all_data.tüketim.notnull()]
test_sub=all_data[all_data.tüketim.isnull()]
test_sub.drop('tüketim',axis=1,inplace=True)
print(test_sub.shape)
print(data_train.shape)

In [ ]:
test_sub

## Model Tuned

In [ ]:
tscv = TimeSeriesSplit(n_splits=3, test_size=2200) 
target=data_train.tüketim
train_labeled=data_train.drop('tüketim',axis=1)
def objective(trial,data=train_labeled,target=target,tscv=tscv):
    
        param = {
            'base_score':0.5,
            'objective':'reg:squarederror',
            'booster':'gbtree', 
            'lambda': trial.suggest_float('lambda', 1e-3, 10),
            'alpha': trial.suggest_float('alpha', 1e-3, 10),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.7,0.8,0.9,1.0]),
            'subsample': trial.suggest_categorical('subsample', [0.5,0.6,0.7,0.8,1.0]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.08,0.009,0.01,0.012,0.014,0.016,0.018,0.001,0.1,0.5]),
            'n_estimators':1500,
            'max_depth': trial.suggest_categorical('max_depth', [3,4,5,7,9,10]),
            'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 500),
            'tree_method':'gpu_hist',
            'enable_categorical':True,
            'early_stopping_rounds':250
        } 

        best_iteration_list=[]
        score_list=[]
        model_list=[]
        fea_importance_list=[]
        tscv_splits = list(tscv.split(data,target))  
        for split_train, split_val in tscv_splits:
            x_train, y_train = data.iloc[split_train], target.iloc[split_train]
            x_val, y_val = data.iloc[split_val], target.iloc[split_val]

        model = xgb.XGBRegressor(**param)  
        model.fit(x_train,y_train,eval_set=[(x_train, y_train), (x_val, y_val)],verbose=0)
        best_iteration=model.best_iteration
        val_pred=model.predict(x_val)
        model_list.append(model)
        score=mean_absolute_percentage_error(y_val, val_pred)
        score_list.append(score)
        fea_importance_list.append(model.feature_importances_)


        trial.set_user_attr('model',model_list)
        trial.set_user_attr('best_iteration',best_iteration)
        trial.set_user_attr("feature_importance", np.mean(fea_importance_list,axis=0))
        trial.set_user_attr("score_list", score_list)

        scores=np.mean(score)
        return scores



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=85)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
gc.collect()

#### BEST PARAMS 
**>params={
'lambda': 5.102807684567904, 
'alpha': 6.511737741485641,
'colsample_bytree': 0.9, 
'subsample': 0.7, 
'learning_rate': 0.012,
'max_depth': 10,
'random_state': 24,
'min_child_weight': 28,
'n_estimators': 1500,
'tree_method':'gpu_hist',
'enable_categorical':True}**


In [ ]:
param={
'lambda': 8.48551751148768, 'alpha': 2.366829085730496, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.08, 'max_depth': 5, 'random_state': 48, 'min_child_weight': 24,
    'n_estimators': 1500,
    'tree_method':'gpu_hist',
        'enable_categorical':True
} 

all_models=[]
for i in [1200, 54,55]:
    param["random_state"] = i
    model = xgb.XGBRegressor(**param)  
    model.fit(train_labeled,target)
    all_models.append(model)

preds = [model.predict(test_sub)  for model in all_models]
mean_preds = np.mean(preds, axis=0)


best=pd.read_csv('/kaggle/input/bestresultgediz33/gediz_33.csv')
result8=pd.read_csv('/kaggle/input/resulte-759/gediz_8.csv')
best['preds']=mean_preds
best['result8']=result8['Dağıtılan Enerji (MWh)']

print(f'8.ay ortalama tüketim :{train[(train.year==2021) &(train.month==8)].tüketim.mean()}')
print(f'8.ay preds ortalama tüketim:{np.mean(mean_preds)}')
importances=model.feature_importances_
features=test_sub.columns
dum=pd.DataFrame(data={'features':features,'importances_rate':importances}).sort_values('importances_rate',ascending=False)
sns.set(rc={'figure.figsize':(30,15)})
g=sns.barplot(x=dum.features,y=dum.importances_rate)
g.set_xticklabels(g.get_xticklabels(), rotation=90);

In [ ]:
submission=pd.read_csv('/kaggle/input/gdz-elektrik-datathon-2023/sample_submission.csv')
submission['Dağıtılan Enerji (MWh)']=mean_preds
submission.set_index('Tarih').to_csv('/kaggle/working/gediz_57.csv')

In [ ]:
plt.plot(best['Dağıtılan Enerji (MWh)'])

In [ ]:
plt.plot(mean_preds)

In [ ]:
best